In [ ]:
from pyo3_dataframe import start_query
import pandas as pd
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
import io
import logging

In [ ]:

def setup_figure() -> go.FigureWidget:
    fig = go.FigureWidget()
    fig.add_bar()
    fig.layout.title = "average price for each fruit"
    fig.layout.xaxis.title = "fruit"
    fig.layout.yaxis.title = "average price"
    fig.layout.yaxis.type = "log"
    return fig

def update_figure(fig: go.FigureWidget, data: tuple[list[str], list[float]]) -> None:
    fig.data[0].x = data[0]
    fig.data[0].y = data[1] 

def df_transformer(result: str) -> tuple[list[str], list[float]]: 
    buf = io.StringIO(result)
    df = pl.read_json(buf)
    # pyarrow's dataframe has different structure than pandas's dataframe
    # calling to_pandas to get pandas's dataframe
    df = df.to_pandas()
    
    x = df["fruit"].values
    y = df["price"].values

    return (x, y)

In [ ]:
class RustLogHandler(logging.Handler):
    def emit(self, record):
        # in Rust, I print out the query execution time 
        # to the standard output prepending the magic word 'CAFEBABE'
        # log format:
        # CAFEBABE query_num query_execution_time
        if record.msg.startswith("CAFEBABE"):
            _, i, time = record.msg.split()
            print("query", i, "execution time:", time)


logging.basicConfig(level=logging.INFO, handlers=[RustLogHandler()])

In [ ]:
fig = setup_figure()
display(fig)

result_consumer = start_query()
for result in result_consumer:
    df = df_transformer(result)
    update_figure(fig, df)